## 3.神经网络优化算法

#### 神经网络的优化过程可以分为两个阶段：

1.先通过前向传播算法计算得到预测数值，并且将预测数值和真实数值做对比得出两者之间的差距。

2.通过反向传播算法，计算损失函数对每一个参数的梯度，再根据梯度和学习率使用梯度下降算法更新每一个参数

#### 梯度下降算法和随机梯度下降算法

梯度下降算法：在海量数据下，要计算所有训练数据的损失函数是非常耗时间的。

随机梯度下降算法：在每一轮的迭代中，随即优化某一条训练数据上的损失函数，甚至可能无法达到局部最优解。

为了综合以上两个的优缺点，在实际应用中选择这两个算法的折中——每次计算一小部分训练数据的损失函数。这一小部分数据称作batch。
通过矩阵运算，每次在一个batch上，优化神经网络的参数并不会比单个数据慢很多。另外一方面，每次使用一个batch可以大大减少收敛所需要的迭代次数，
同时可以使收敛到的结果更加接近梯度下降的结果。

#### 实现过程

batch_size = n

##### 每次选取一小部分数据作为当前的数据来执行反向传播算法

x = tf.placehoulder(tf.float32,shape = (batch_size,2),name = 'x_input')
y_ = tf.placeholder(tf.float32,shape = (batch_size,1),name = 'y_input')

##### 定义神经网络结构和优化算法

loss = ....
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

##### 训练神经网络

with tf.Session() as sess:
##### 参数初始化       
##### 迭代更新参数   
    for i in range(STEPS):
  ##### 准备batch_size 个训练数据，一般所有训练数据随机打乱之后再选取可以得到更好的优化效果
        current_x,current_y = .....
        sess.run(train_step,feed_dict = {x:current_X, y_:current_y})

## 4.神经网络进一步优化

### 4.1学习率的设置

#### 指数衰减法——tf.train.exponential_decay函数

通过这个函数，可以先使用较大的学习率来得到一个比较优的解，然后随着迭代的继续逐步减小学习率，使模型在训练后期更加稳定。

它实现了以下代码的功能：

decayed_learning_rate = learning_rate*decay_Rate^(global_step/decay_steps)

decayed_learning:每一轮优化时候使用的学习率
    
learning_rate:事先设定的初始学习率
    
decay_rate:衰减系数
    
decay_steps:衰减速度


#### 实现代码

global_step = tf.Variable(0)


##### 通过exponential_decay函数生成学习率
##### 基础学习率0.1，衰减率0.96，衰减速度100

learning_rate = tf.train.exponential_decay(0.1,global_step,100,0.96,staircase = True)


##### 使用指数衰减的学习率，在minimize函数中传入global_step将自动更新global_step参数，从而使得学习率也得到相应的更新

learning_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
##### 上面这段代码，每次训练100轮以后学习率乘以0.96.一般来说学习率、衰减系数和衰减速度都是根据经验设置的。


### 4.2过拟合问题

正则化：思想是在损失函数中加入刻画模型复杂程度的指标，使模型不能任意拟合训练数据中的随机噪声。

#### 带有L2正则化的损失函数

w = tf.Variable(tf.random_normal([2,1], stddev = 1,seed = 1)
                
y = tf.matmul(x,w)

loss = tf.reduce_mean(tf.square(y_ - y))+tf.contrib.layers.l2_regularizer(lambda)(w)
                

loss 定义为损失函数。由两部分组成，第一部分是均方差损失函数，第二部分是正则化。lambda 参数表示正则化的权重

#### L1,L2函数样例

In [19]:
import tensorflow as tf
weights = tf.constant([[1.0,-2.0],[-3.0,4.0]])
with tf.Session() as sess:
    
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(weights)))
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weights)))

5.0
7.5


#### 通过集合计算一个五层神经网络带L2正则化的损失函数计算方法

In [2]:
import tensorflow as tf

获取一层神经网络边上的权重，并且将这个权重的 L2正则化损失加入名称‘losses’的集合中

In [3]:
def get_weight(shape,lambda_):
    var = tf.Variable(tf.random_normal(shape),dtype = tf.float32)
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(lambda_)(var))
    return var    

x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))
batch_size = 8

定义每一层网络中节点的个数

In [4]:
layer_dimension = [2,10,10,10,1]

神经网络的层数

In [5]:
n_layers = len(layer_dimension)

这个变量维护向前传播时最深的节点，开始的时候是输入层

In [6]:
cur_layer = x

当前层节点个数

In [7]:
in_dimension = layer_dimension[0]

通过一个循环来生成5层全连接的神经网络结构

layer_dimension[i]是下一层节点个数

生成当前层中权重的变量，并且将这个变量的L2正则化损失加入计算图上的集合

使用rel函数

进入下一层之前将下一层的节点个数更新为当前层节点个数

In [8]:
for i in range(1,n_layers):
    out_dimension = layer_dimension[i]
    weight = get_weight([in_dimension,out_dimension],0.001)
    bias = tf.Variable(tf.constant(0.1,shape = [out_dimension]))
    cur_layer = tf.nn.relu(tf.matmul(cur_layer,weight)+bias)
    in_dimension = layer_dimension[i]


在定义神经网络向前传播胡同时已经将所有胡L2正则化损失加入了图上的集合这里只需要计算刻画模型在训练数据上表现的损失函数

In [9]:
mse_loss = tf.reduce_mean(tf.square(y_ - cur_layer))

将均方差损失函数加入损失集合

In [10]:
tf.add_to_collection('losses',mse_loss)

get_collection返回一个列表，这个列表是所有这个集合中的元素，在这个样例中这些元素就是损失函数的不同部分，
将他们加起来就可得到最终的损失函数。

In [11]:
loss = tf.add_n(tf.get_collection('losses'))

### 4.3滑动平均模型

TF通过提供tf.train.ExponentialMovingAverage来实现滑动平均模型

#### 代码实现

In [12]:
import tensorflow as tf

定义一个变量用于计算滑动平均，这个变量的初始数值是0。注意这里是手动指定了变量的类型是tf.float32,因此所有需要计算滑动平均的变量必须是实数

In [13]:
v1 = tf.Variable(0,dtype = tf.float32)

这里的step变量模拟神经网络中迭代的轮数，可以用于动态控制衰减率

In [14]:
step = tf.Variable(0,trainable = False )

定义一个滑动平均类型（class）。

初始化给定衰减率（0.99）和控制衰减的变量（step）。

In [15]:
ema = tf.train.ExponentialMovingAverage(0.99,step)

定义一个更新变量滑动平均的操作。

这里需要给定一个列表，每次执行操作的时候这个列表都会被更新。

In [18]:
maintain_averages_op = ema.apply([v1])

In [17]:
with tf.Session() as sess:
    # 初始化所有变量
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # 通过ema.average(v1)获取滑动平均之后变量的取值。在取值之后变量v1的数值和v1的滑动平均都是0.
    print(sess.run([v1,ema.average(v1)]))
    # 更新v1的数值到5
    sess.run(tf.assign(v1,5)) 
    # 更新v1的滑动平均值。衰减率是min{0.99,(1+step)/(10+step) = 0.1} = 0.1
    # 所以v1的滑动平均会被更新为0.1x0+0.9x5 = 4.5
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    # 更新step的数值是10000     
    sess.run(tf.assign(step,10000))
    # 更新v1的数值是10
    sess.run(tf.assign(v1,10))
    # 更新v1的滑动平均值。衰减率是min{0.99，（1+step）/（10+step） = 0.999} = 0.99
    # 所以v1的滑动平均会被更新为0.99x4.5+0.01x10 = 4.555
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))
    # 再次更新滑动平均值，得到新的滑动平均值为0.99x4.555+0.01x10 = 4.60945
    sess.run(maintain_averages_op)
    print(sess.run([v1,ema.average(v1)]))

[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.555]
[10.0, 4.60945]
